## DINOv3 Fine-tuning for Image Classification

In [1]:
!pip install -q trackio git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.5/838.5 kB 41.3 MB/s eta 0:00:00


## Dataset

We will do a very small run on food101 dataset.

In [7]:
from datasets import load_dataset

ds = load_dataset("ethz/food101")

train_ds = ds["train"]
train_ds = train_ds.shuffle().train_test_split(test_size=0.9)["train"]
val_ds = ds["validation"].shuffle().train_test_split(test_size=0.9)["train"]

In [8]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 7575
})

In [9]:
val_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 2525
})

In [10]:
num_classes = train_ds.features["label"].num_classes
id2label = {i: name for i, name in enumerate(train_ds.features["label"].names)}
label2id = {v: k for k, v in id2label.items()}
print(f"Classes: {num_classes}")

Classes: 101


## Load Model

This model doesn't come with a head, so we need to write the headed model class.

In [11]:
import torch.nn as nn
import torch
from transformers import AutoImageProcessor, AutoModel, get_cosine_schedule_with_warmup

MODEL_NAME = "facebook/dinov3-vith16plus-pretrain-lvd1689m"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


image_processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
backbone = AutoModel.from_pretrained(MODEL_NAME)

hidden_size = getattr(backbone.config, "hidden_size", None)

class DinoV3Linear(nn.Module):
    def __init__(self, backbone: AutoModel, hidden_size: int, num_classes: int, freeze_backbone: bool = True):
        super().__init__()
        self.backbone = backbone
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False
            self.backbone.eval()

        self.head = nn.Linear(hidden_size, num_classes)

    def forward(self, pixel_values):
        outputs = self.backbone(pixel_values=pixel_values)
        last_hidden = outputs.last_hidden_state
        cls = last_hidden[:, 0]
        logits = self.head(cls)
        return logits

model = DinoV3Linear(backbone, hidden_size, num_classes, freeze_backbone=True).to(device) # we only train the head

preprocessor_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Write the data collator to batch inputs and dataloaders for training.

In [12]:
from dataclasses import dataclass
from PIL import Image
import numpy as np
import torch
from transformers import AutoImageProcessor

@dataclass
class Collator:
    processor: AutoImageProcessor

    def __call__(self, batch):
        raw_images = [x["image"] for x in batch]
        labels = torch.tensor([x["label"] for x in batch], dtype=torch.long)

        rgb_images = []
        # there's grayscale images in the dataset
        for im in raw_images:
            if isinstance(im, Image.Image):
                rgb_images.append(im.convert("RGB"))
                continue

        inputs = self.processor(images=rgb_images, return_tensors="pt")
        return {"pixel_values": inputs["pixel_values"], "labels": labels}

collate_fn = Collator(image_processor)

In [13]:
from torch.utils.data import DataLoader
import os

BATCH_SIZE = 8
NUM_WORKERS = min(8, os.cpu_count() or 2)

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=collate_fn,
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=collate_fn,
)

## Training

Find config below.

In [14]:
import math
import random
from typing import List, Dict, Any


EPOCHS = 5
LR = 5e-4
WEIGHT_DECAY = 1e-4
WARMUP_RATIO = 0.05
CHECKPOINT_DIR = "./checkpoints_dinov3_food101"
EVAL_EVERY_STEPS = 100

optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR, weight_decay=WEIGHT_DECAY)
total_steps = EPOCHS * math.ceil(len(train_loader))
warmup_steps = int(WARMUP_RATIO * total_steps)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
criterion = nn.CrossEntropyLoss()

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

/tmp/ipython-input-593493728.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


In [15]:
os.makedirs("./checkpoints_dinov3_food101")

We need to evaluate during training.

In [16]:
def evaluate() -> Dict[str, float]:
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0
    with torch.no_grad():
        for batch in val_loader:
            pixel_values = batch["pixel_values"].to(device, non_blocking=True)
            labels = batch["labels"].to(device, non_blocking=True)
            logits = model(pixel_values)
            loss = criterion(logits, labels)
            loss_sum += loss.item() * labels.size(0)
            preds = logits.argmax(dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return {
        "val_loss": loss_sum / max(total, 1),
        "val_acc": correct / max(total, 1),
    }

Let's write the training loop. We'll also use trackio for experiment tracking.

In [17]:
import trackio

best_acc = 0.0
global_step = 0

trackio.init(project="dinov3", config={
            "epochs": EPOCHS,
            "learning_rate": LR,
            "batch_size": BATCH_SIZE
        })

for epoch in range(1, EPOCHS + 1):
    model.train()
    model.backbone.eval()  # comment out if you want to train the whole model

    running_loss = 0.0
    for i, batch in enumerate(train_loader, start=1):
        pixel_values = batch["pixel_values"].to(device, non_blocking=True)
        labels = batch["labels"].to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(pixel_values)
        loss = criterion(logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()
        global_step += 1

        if global_step % EVAL_EVERY_STEPS == 0:
            metrics = evaluate()
            print(
                f"[epoch {epoch} | step {global_step}] "
                f"train_loss={running_loss / EVAL_EVERY_STEPS:.4f} "
                f"val_loss={metrics['val_loss']:.4f} val_acc={metrics['val_acc']*100:.2f}%"
            )
            running_loss = 0.0

            trackio.log(
                    {
                        "epoch": epoch,
                        "val_acc": best_acc,
                    }
                )

            if metrics["val_acc"] > best_acc:
                best_acc = metrics["val_acc"]
                ckpt_path = os.path.join(CHECKPOINT_DIR, f"best_acc_{best_acc:.4f}.pt")
                torch.save(
                    {
                        "model_state_dict": model.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "scheduler_state_dict": scheduler.state_dict(),
                        "config": {
                            "model_name": MODEL_NAME,
                            "num_classes": num_classes,
                        },
                        "step": global_step,
                        "epoch": epoch,
                    },
                    ckpt_path,
                )


    metrics = evaluate()
    print(
        f"END EPOCH {epoch}: val_loss={metrics['val_loss']:.4f} val_acc={metrics['val_acc']*100:.2f}% "
        f"(best_acc={best_acc*100:.2f}%)"
    )
    trackio.finish()

* Running on public URL: https://3669a91d39321f7f86.gradio.live
* Trackio project initialized: dinov3
* Trackio metrics logged to: /root/.cache/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "dinov3"
* or by running in Python: trackio.show(project="dinov3")
[epoch 1 | step 100] train_loss=4.4878 val_loss=4.0990 val_acc=50.77%
[epoch 1 | step 200] train_loss=3.4722 val_loss=2.5605 val_acc=83.72%
[epoch 1 | step 300] train_loss=1.9046 val_loss=1.2049 val_acc=87.09%
[epoch 1 | step 400] train_loss=1.0664 val_loss=0.7385 val_acc=89.78%
[epoch 1 | step 500] train_loss=0.7269 val_loss=0.5500 val_acc=90.30%


/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


[epoch 1 | step 600] train_loss=0.6400 val_loss=0.4473 val_acc=91.92%
[epoch 1 | step 700] train_loss=0.5444 val_loss=0.3916 val_acc=92.44%
[epoch 1 | step 800] train_loss=0.5084 val_loss=0.3506 val_acc=92.08%


KeyboardInterrupt: 

In [ ]:
!trackio show

Let's infer with the model, I have a few in the wild images.

In [19]:
import torch
from PIL import Image
from typing import List, Dict


model.eval()

images = ["/content/pizza.jpg", "/content/spaghetti.JPG"]

pil_images = [Image.open(p).convert("RGB") for p in images]
inputs = image_processor(images=pil_images, return_tensors="pt").to(device)

with torch.no_grad():
  logits = model(inputs["pixel_values"])

# take top 2 classes
probs = logits.softmax(dim=-1)
scores, indices = probs.topk(2, dim=-1)

results = []
for path, idxs, scs in zip(images, indices, scores):
    preds = [
        {"label_id": int(i.item()),
          "label": id2label.get(int(i.item()), f"class_{int(i)}"),
          "score": float(s.item())}
        for i, s in zip(idxs, scs)
    ]
    results.append({"image": path, "topk": preds})


The model predicts correctly, which is expected given we only trained head with the great backbone frozen, it learned very fast. Feel free to try with more challenging use cases.

In [20]:
results

[{'image': '/content/pizza.jpg',
  'topk': [{'label_id': 76, 'label': 'pizza', 'score': 0.7595003843307495},
   {'label_id': 35, 'label': 'escargots', 'score': 0.013227012008428574}]},
 {'image': '/content/spaghetti.JPG',
  'topk': [{'label_id': 91,
    'label': 'spaghetti_carbonara',
    'score': 0.6622196435928345},
   {'label_id': 90,
    'label': 'spaghetti_bolognese',
    'score': 0.18182380497455597}]}]